In [1]:
import os
os.chdir('/home/azureuser/cloudfiles/code/Users/oviemunooboro/Product_recommendation_system')

from dataclasses import dataclass
from pathlib import Path
from src.utils.commons import read_yaml,create_directories
from src.cloud_storage.azure_blob_storage import AzureDatastore
import os
import sys
from datetime import datetime
from src.logger import logging
import pandas  as pd
from src.constants import *
import time
from datetime import datetime
from src.utils.commons import unzip_files
from glob import glob
import great_expectations as gx
import great_expectations.expectations as gxe
from great_expectations.exceptions import GreatExpectationsError
from src.utils.commons import save_json

[2024-12-11 20:09:16,990 ] 161 numexpr.utils - INFO - NumExpr defaulting to 4 threads.
[2024-12-11 20:09:17,589 ] 103 great_expectations._docs_decorators - INFO - Skipping registering function get_context because it does not have a class
[2024-12-11 20:09:18,757 ] 97 great_expectations._docs_decorators - INFO - Skipping registering function DataSourceManager._register_add_datasource.<locals>.crud_method_info because it is a closure
[2024-12-11 20:09:18,758 ] 97 great_expectations._docs_decorators - INFO - Skipping registering function DataSourceManager._register_update_datasource.<locals>.crud_method_info because it is a closure
[2024-12-11 20:09:18,759 ] 97 great_expectations._docs_decorators - INFO - Skipping registering function DataSourceManager._register_add_or_update_datasource.<locals>.crud_method_info because it is a closure
[2024-12-11 20:09:18,760 ] 97 great_expectations._docs_decorators - INFO - Skipping registering function DataSourceManager._register_delete_datasource.<loc

In [7]:
# enttity

@dataclass
class DatavalidationConfig:
    status_file : Path


In [3]:
# configuration manager

class ConfigurationManager:

    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):

        # initialiazing filepath

        self.config = read_yaml(str(config_filepath))
        self.schema = read_yaml(str(schema_filepath))
        self.params = read_yaml(str(params_filepath))

    def get_data_validation_config(self):

        config = self.config.DATA_VALIDATION

        create_directories([config.root_dir])

        data_validation_config = DatavalidationConfig(
                        status_file = config.status_file
        )

        return data_validation_config



In [8]:
class DataValidation:

    def __init__(self, config=DatavalidationConfig, source_folder='Artifacts', 
                 data_source_name='ingested data local file_system', asset_name='ingested customer interaction data', 
                 data_directory='ingested_data', batch_definition_name='customer data batch', 
                 expectation_suite_name='ingested customer data expectation suites',validation_definition_name = "ingested_data_validation_definition"):
        self.config = config
        self.source_folder = source_folder
        self.data_source_name = data_source_name
        self.asset_name = asset_name
        self.data_directory = data_directory
        self.batch_definition_name = batch_definition_name
        self.expectation_suite_name = expectation_suite_name
        self.validation_definition_name = validation_definition_name

        # Access numerical and categorical columns from config
        self.numerical_columns = config.columns.numerical_columns
        self.categorical_columns = config.columns.categorical_columns

        self.context = gx.get_context(mode='file')
        

    def validation_setup(self):
        try:
            logging.info('setting up validation')

            # Check if the data source already exists
            try:
                data_source = self.context.data_sources.get(self.data_source_name)
                logging.info(f'Data source "{self.data_source_name}" already exists.')
            except KeyError:
                # If the data source doesn't exist, create it
                data_source = self.context.data_sources.add_spark_filesystem(
                    name=self.data_source_name, base_directory=self.source_folder
                )
                logging.info(f'Data source "{self.data_source_name}" added.')

            # Check if the asset exists or create it
            try:
                data_asset = data_source.get_asset(name=self.asset_name)
                logging.info(f'Asset "{self.asset_name}" already exists.')
            except LookupError:
                # If asset doesn't exist, create it
                data_asset = data_source.add_directory_csv_asset(
                    name=self.asset_name, data_directory=self.data_directory, header=True
                )
                logging.info(f'Asset "{self.asset_name}" added.')

            # Add or get the batch definition
            try:
                batch_definition = data_asset.get_batch_definition(self.batch_definition_name)
                logging.info(f'Data source "{self.batch_definition_name}" already exists.')
            except KeyError:
                    batch_definition = data_asset.add_batch_definition_whole_directory(
                        self.batch_definition_name
                    )
                    logging.info(f'Batch definition "{self.batch_definition_name}" added.')

        except (Exception, GreatExpectationsError) as e:
            logging.error(f'Error occurred: {e}')
            # Fetch existing batch definition if there's an error
            data_source = self.context.data_sources.get(self.data_source_name)
            data_asset = data_source.get_asset(name=self.asset_name)
            batch_definition = data_asset.get_batch_definition(self.batch_definition_name)

        # Return the batch definition
        return batch_definition

    def expectation_suite(self):
        try:
            # Check if the expectation suite already exists
            try:
                expectation_suite = self.context.suites.get(self.expectation_suite_name)
                logging.info(f'Expectation suite "{self.expectation_suite_name}" already exists.')
            except Exception:
                # If the expectation suite doesn't exist, create it
                expectation_suite = self.context.suites.add(
                    gx.ExpectationSuite(name=self.expectation_suite_name)
                )
                logging.info(f'Expectation suite "{self.expectation_suite_name}" created.')
        except:
            # If an error occurs (e.g., the suite exists but could not be fetched), delete and recreate
            self.context.suites.delete(name=self.expectation_suite_name)
            expectation_suite = self.context.suites.add(
                gx.ExpectationSuite(name=self.expectation_suite_name)
            )
            logging.info(f'Expectation suite "{self.expectation_suite_name}" recreated.')

        # Define the expectations
        expectations = [
            *[gxe.ExpectColumnToExist(column=x, column_index=0) for x in ['price','product_id','user_id','category_id', 'event_time',
                                                                          'event_type','category_code', 'brand','user_session']],

            gxe.ExpectColumnDistinctValuesToBeInSet(column='event_type', value_set=['cart','remove_from_cart','purchase','view']),

            gxe.ExpectCompoundColumnsToBeUnique(column_list=['price','product_id','user_id','category_id', 'event_time','event_type',
                                                             'category_code', 'brand','user_session']),

            *[gxe.ExpectColumnValuesToNotBeNull(column=x) for x in ['user_id','event_time','product_id','category_id','event_type','user_session']],

            *[gxe.ExpectColumnValuesToBeOfType(column=x, type_='StringType') for x in ['user_id','product_id','category_id','user_session','event_type']],

            gxe.ExpectColumnValuesToBeInTypeList(column='price', type_list=['DoubleType','FloatType','IntegerType']),
            gxe.ExpectColumnValuesToBeOfType(column='event_time', type_='TimestampType')
        ]

        # Add expectations to the suite
        for expectation in expectations:
            expectation_suite.add_expectation(expectation)

        return expectation_suite

    def validate(self):
        batch_definition = self.validation_setup()
        # Validate Batch using Expectation Suite.
        expectation_suite = self.expectation_suite()

        try:
            # Check if validation definition already exists
            validation_definition = self.context.validation_definitions.get(self.validation_definition_name)
            logging.info(f'Validation definition "{self.validation_definition_name}" already exists.')
        except Exception:
            # If the validation definition doesn't exist, create it
            validation_definition = gx.ValidationDefinition(
                data=batch_definition, suite=expectation_suite, name=self.validation_definition_name
            )
            logging.info(f'Validation definition "{self.validation_definition_name}" created.')

        validation_results = validation_definition.run().to_json_dict()

        save_json(filepath=Path(self.config.status_file),data=validation_results)
       

In [9]:
man = ConfigurationManager()
config = man.get_data_validation_config()
val = DataValidation(config=config)

vali = val.validate()

[2024-12-11 20:13:39,799 ] 39 root - INFO - Yaml file:  config/config.yaml loaded suscessfully
[2024-12-11 20:13:39,821 ] 39 root - INFO - Yaml file:  schema.yaml loaded suscessfully
[2024-12-11 20:13:39,840 ] 39 root - INFO - Yaml file:  params.yaml loaded suscessfully
[2024-12-11 20:13:39,848 ] 62 root - INFO - File directory create at : Artifacts/data_validation
[2024-12-11 20:13:39,926 ] 209 great_expectations.data_context.data_context.file_data_context - INFO - FileDataContext loading fluent config
[2024-12-11 20:13:39,930 ] 191 great_expectations.datasource.fluent.config - INFO - Loading 'datasources' ->
[{'assets': [...],
  'base_directory': 'Artifacts',
  'id': '7f9d031e-51ab-4845-8a4c-5726d5b986d1',
  'name': 'ingested data local file_system',
  'type': 'spark_filesystem'}]
[2024-12-11 20:13:40,001 ] 2429 great_expectations.data_context.data_context.abstract_data_context - INFO - Loaded 'ingested data local file_system' from fluent config
[2024-12-11 20:13:40,003 ] 25 root - I

[2024-12-11 20:13:40,431 ] 1213 great_expectations.expectations.expectation - INFO - _get_default_value called with key "table", but it is not a known field
[2024-12-11 20:13:40,433 ] 1213 great_expectations.expectations.expectation - INFO - _get_default_value called with key "table", but it is not a known field
[2024-12-11 20:13:40,436 ] 1213 great_expectations.expectations.expectation - INFO - _get_default_value called with key "table", but it is not a known field
[2024-12-11 20:13:40,438 ] 1213 great_expectations.expectations.expectation - INFO - _get_default_value called with key "table", but it is not a known field
[2024-12-11 20:13:40,440 ] 1213 great_expectations.expectations.expectation - INFO - _get_default_value called with key "table", but it is not a known field
[2024-12-11 20:13:40,442 ] 1213 great_expectations.expectations.expectation - INFO - _get_default_value called with key "table", but it is not a known field
[2024-12-11 20:13:40,444 ] 1213 great_expectations.expecta

24/12/11 20:13:40 WARN CacheManager: Asked to cache already cached data.
/anaconda/envs/azureml_py38/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


Calculating Metrics:   0%|          | 0/48 [00:00<?, ?it/s]

24/12/11 20:13:40 WARN MemoryStore: Not enough space to cache rdd_13_0 in memory! (computed 16.1 MiB so far)
24/12/11 20:13:40 WARN MemoryStore: Not enough space to cache rdd_13_2 in memory! (computed 17.0 MiB so far)
24/12/11 20:13:40 WARN MemoryStore: Not enough space to cache rdd_13_3 in memory! (computed 8.4 MiB so far)
24/12/11 20:13:40 WARN MemoryStore: Not enough space to cache rdd_13_4 in memory! (computed 31.5 MiB so far)
24/12/11 20:13:41 WARN MemoryStore: Not enough space to cache rdd_13_6 in memory! (computed 15.0 MiB so far)
24/12/11 20:13:41 WARN MemoryStore: Not enough space to cache rdd_13_5 in memory! (computed 30.0 MiB so far)
24/12/11 20:13:41 WARN MemoryStore: Not enough space to cache rdd_13_7 in memory! (computed 22.9 MiB so far)
24/12/11 20:13:41 WARN MemoryStore: Not enough space to cache rdd_13_10 in memory! (computed 16.0 MiB so far)
24/12/11 20:13:41 WARN MemoryStore: Not enough space to cache rdd_13_13 in memory! (computed 16.7 MiB so far)
24/12/11 20:13:41 

[2024-12-11 20:15:22,196 ] 1213 great_expectations.expectations.expectation - INFO - _get_default_value called with key "table", but it is not a known field
[2024-12-11 20:15:22,198 ] 1213 great_expectations.expectations.expectation - INFO - _get_default_value called with key "table", but it is not a known field
[2024-12-11 20:15:22,199 ] 1213 great_expectations.expectations.expectation - INFO - _get_default_value called with key "table", but it is not a known field
[2024-12-11 20:15:22,200 ] 1213 great_expectations.expectations.expectation - INFO - _get_default_value called with key "table", but it is not a known field
[2024-12-11 20:15:22,203 ] 1213 great_expectations.expectations.expectation - INFO - _get_default_value called with key "table", but it is not a known field
[2024-12-11 20:15:22,204 ] 1213 great_expectations.expectations.expectation - INFO - _get_default_value called with key "table", but it is not a known field
[2024-12-11 20:15:22,206 ] 1213 great_expectations.expecta

[2024-12-11 20:15:22,513 ] 111 root - INFO - json file saved at: Artifacts/data_validation/status.json


In [ ]:
print(vali)

None
